In [1]:
from datasets import load_dataset

dataset64 = load_dataset("json", data_files="./comments_guns_with_scores_above_quantile_0.90_train_64.json")

c:\Users\ysong\AppData\Local\anaconda3\envs\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset json (C:/Users/ysong/.cache/huggingface/datasets/json/default-7be31e0ae052b38a/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)
100%|██████████| 1/1 [00:00<00:00, 177.16it/s]


In [2]:
dataset64 = dataset64["train"]

In [3]:
dataset64["subreddit"][10:30]

['progun',
 'GunsAreCool',
 'progun',
 'GunsAreCool',
 'progun',
 'GunsAreCool',
 'progun',
 'progun',
 'GunsAreCool',
 'GunsAreCool',
 'GunsAreCool',
 'progun',
 'GunsAreCool',
 'GunsAreCool',
 'GunsAreCool',
 'GunsAreCool',
 'GunsAreCool',
 'GunsAreCool',
 'GunsAreCool',
 'progun']

In [4]:
dataset64[0]

{'link_id': 't3_aud8x',
 'subreddit': 'guncontrol',
 'retrieved_on': 1426196154.0,
 'score': 6.0,
 'body': "Nothing. I was just pointing that out to demonstrate that, although Jesse Ventura is cool, he's not right about everything.",
 'gilded': 0,
 'id': 'c0jgzwt',
 'predicted_community': 3,
 'subreddit_id': 't5_2rdus',
 'created_utc': 1264557266,
 'controversiality': 0,
 'edited': -1.0,
 'parent_id': 't1_c0jgqpd',
 'language': 'en',
 'distinguished': None,
 'body_cleaned': "nothing . i was just pointing that out to demonstrate that , although jesse ventura is cool , he 's not right about everything .",
 'author': 'HAZod'}

In [8]:
df = dataset64.to_pandas()
predicted_community = df["predicted_community"].value_counts()
predicted_community

predicted_community
3    2815
2    1526
0    1376
Name: count, dtype: int64

In [9]:
148 / (148 + 654 + 911)

0.08639813193228255

### Create count-balanced dataset

In [10]:
sorted_df = df.groupby('predicted_community').apply(lambda x: x.nlargest(1000, 'score')).reset_index(drop=True)
sorted_df.value_counts("predicted_community")

predicted_community
0    1000
2    1000
3    1000
Name: count, dtype: int64

In [20]:
sorted_df.to_json('train_64_count_balanced.json', orient='records', lines=True)

### Create sum-balanced dataset

In [26]:
predicted_communities = df['predicted_community'].unique()
predicted_communities

array([3, 0, 2], dtype=int64)

In [27]:
dataframes = {}
for community in predicted_communities:
    community_df = df[df['predicted_community'] == community].copy()
    community_df = community_df.sort_values('score', ascending=False).reset_index(drop=True)
    dataframes[f'df_pc_{community}'] = community_df

In [33]:
dataframes.keys()

dict_keys(['df_pc_3', 'df_pc_0', 'df_pc_2'])

In [35]:
df.groupby('predicted_community')['score'].sum()

predicted_community
0    51623.0
2    64672.0
3    45050.0
Name: score, dtype: float64

In [41]:
dataframes['df_pc_3'].head(10)[["predicted_community", "score"]]
    

,predicted_community,score
0,3,142.0
1,3,135.0
2,3,127.0
3,3,109.0
4,3,96.0
5,3,88.0
6,3,82.0
7,3,78.0
8,3,77.0
9,3,76.0


In [49]:
import pandas as pd

def extract_examples(df, bar=30000, bias=50):
    examples = []
    cumulative_score = 0
    
    for index, row in df.iterrows():
        examples.append(row)
        cumulative_score += row['score']
        
        if cumulative_score >= bar - bias:
            break
    
    return pd.DataFrame(examples)

In [52]:
new_dataframes = {}

for key in dataframes.keys():
    new_df = extract_examples(dataframes[key])
    new_dataframes["new_" + key] = new_df

In [54]:
for i, key in enumerate(new_dataframes.keys()):
    print(new_dataframes[key].groupby('predicted_community')['score'].sum())

predicted_community
3    29962.0
Name: score, dtype: float64
predicted_community
0    29974.0
Name: score, dtype: float64
predicted_community
2    29984.0
Name: score, dtype: float64


In [57]:
concatenated_df = pd.concat(new_dataframes.values(), ignore_index=True)
concatenated_df.to_json('train_64_sum_balanced.json', orient='records', lines=True)

## Top sentencens

In [14]:
filtered_df = df[df['subreddit'].isin(['progun', 'liberalgunowners', 'guncontrol'])]

In [18]:
sorted_df = filtered_df.sort_values('score', ascending=False)
top_10_sentences = sorted_df.head(10)[['body', 'subreddit']]

In [19]:
for index, row in top_10_sentences.iterrows():
    print("Subreddit:", row['subreddit'])
    print("Sentence:", row['body'])
    print()



Subreddit: progun
Sentence: As a federal LEO, the very idea of confiscating guns is laughable. I swore an oath to the constitution, not to Beto or any other politician.

Subreddit: progun
Sentence: We must ban this horrible weapon of war

Subreddit: liberalgunowners
Sentence: This is also why I love Hickok45. He's just an all-around good guy. One of the best representatives out there for the pro-2A community.

Subreddit: liberalgunowners
Sentence: for good reason.


god she sucked so much

Subreddit: liberalgunowners
Sentence: He is the worst parts of a Democrat and the worst parts of a Republican, rolled into one. An authoritarian, through and through.

Subreddit: progun
Sentence: "You're pretty stupid to not just have armed security like me you filthy peasants." That's all I read.

Subreddit: progun
Sentence: Meanwhile in England, an old man is being charged with murder after he killed one of the men breaking into his home

Subreddit: liberalgunowners
Sentence: This is part of why he